# Vessel Route Optimization

In [7]:
import math
import numpy as np
from numpy import random
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

## Semi-random vectors generation

In [2]:
wind_degrees = []
days = 10
max_change = 0.3
#random.seed(1234)

for day in range(days):
    if day == 0:
        wind_degrees.append(random.randint(0,360))
    else:
        last_day = wind_degrees[day-1]
        wind_degrees.append(random.randint(last_day*(1-max_change),last_day*(1+max_change)))
        
wind_speed = list(random.randint(0,28,(1,days)).flatten())

print("Wind direction (degrees):", wind_degrees)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [150, 142, 149, 108, 127, 135, 135, 152, 142, 145]
Wind speed (m/s): [26, 25, 7, 6, 16, 17, 14, 1, 3, 9]


## Multiple array problem

In [3]:
def random_space(max_value, days=5, positions=1, day_max_change=0.3, position_max_change=0.2, seed=None):    
    random.seed(seed)
    degrees_matrix = []
    wind_degrees = []
    
    for day in range(days):
        wind_degrees.append(random.randint(1,max_value))
    degrees_matrix.append(wind_degrees) 

    for position in range(0,positions-1):
        position_degree = []
        for day in range(days):
            same_day_other_position = abs(degrees_matrix[position-1][day])  
            if same_day_other_position != 0:
                position_degree.append(random.randint(same_day_other_position*(1-position_max_change),same_day_other_position*(1+position_max_change)))
            else: 
                position_degree.append(random.randint(1,max_value))
        degrees_matrix.append(position_degree)
    
    return degrees_matrix

days = 5
positions = 3
day_max_change = 0.3
position_max_change = 0.3
seed = 126

wind_direction = random_space(360, days, positions, day_max_change, position_max_change)
wind_speed = random_space(27, days, positions, day_max_change, position_max_change)

print("Wind direction (degrees):", wind_direction)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [[79, 38, 185, 305, 119], [94, 44, 200, 365, 103], [98, 46, 135, 371, 145]]
Wind speed (m/s): [[21, 10, 1, 12, 18], [22, 10, 0, 11, 16], [14, 7, 0, 10, 20]]


# Open_sail function

In [6]:
#CONSTANTS TO BE DETERMINED
min_wind_angle = 60 
max_wind_angle = 300 
min_wind_strength = 10

def open_sail(wind_angle, wind_strength):
    open_sail_bool = False 
    
    if wind_angle > min_wind_angle and wind_angle < max_wind_angle and wind_strength > min_wind_strength :
        open_sail_bool = True
        
    #EXCLUDING CRIETRIA FOR THE BOAT_SPEED FUNCTION NOT TO EXPLODE
    if wind_angle > 80 and wind_angle < 120 : open_sail_bool = False
    if wind_angle > 260 and wind_angle < 300 : open_sail_bool = False
    if wind_angle==10 or wind_angle ==190 : open_sail_bool = False


    return open_sail_bool


print("We open the sail : ", open_sail(80,40))

We open the sail :  True


# Apparent wind function for boat speed

In [5]:
# 0<a0<180

def boat_speed(a0,VW,eta):

    f = lambda a: math.sin(math.pi*a0/180) * math.sin(math.pi*a/180) * math.pow((math.sin(math.pi*a/180/2) / math.sin(math.pi*a0/180-math.pi*a/180)), 2)-VW * eta
    a = fsolve(f, [0])[0]
    #print("a:", a)
    VB = VW * (math.sin(math.pi*(a0-a)/180)/math.sin(math.pi*a/180))
    #print("a0:",a0,"and VB:", VB)
    
    return VB

# Problem in 1 dimension

In [62]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 30 #miles
positions = 6 #nb of squares between A and B
eta = 0.001
motor_speed = 15 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle = random_space(360,1,positions) #in degrees
wind_speed = random_space(30,1,positions) #in mph
square_length = total_distance/positions



#defining the lists to be used
list_sail = []
list_boat_speed = []
list_co2 = []
list_time = []

#flatten the list of list into a simple list
wind_angle = [item for sublist in wind_angle for item in sublist]
wind_speed = [item for sublist in wind_speed for item in sublist]

#evaluating if we open the sails for each position
for i in range(positions):
    list_sail.append(open_sail(wind_angle[i],wind_speed[i]))
    
#determining the boat speeds depending on whether we use sails on motor   
for i in range(positions):
    if list_sail[i]==True:
        list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))
        
    else:
        list_boat_speed.append(motor_speed)
        
#CO2 emissions calculations
for i in range(positions):
    if list_sail[i]==True:
        list_co2.append(0)
        
    else:
        list_co2.append(motor_emission_per_mile*square_length)
        
total_emissions = sum(list_co2)
        
#determining the time needed for the journey
for i in range(positions):
    list_time.append(square_length/list_boat_speed[i])
        
total_time = sum(list_time)        
        
    
    
#printing results
print("The squares length is", square_length)
print("Wind angles are :", wind_angle)
print("Wind speeds are :", wind_speed)
print("We open the sails :", list_sail)
print("Boat speeds are :", list_boat_speed)
print("The emissions for each square are:", list_co2)
print("The total emissions are:", total_emissions)
print("The time need for each square is:", list_time)
print("The total time needed is:", total_time)

The squares length is 5.0
Wind angles are : [165, 161, 177, 153, 155, 158]
Wind speeds are : [11, 12, 8, 13, 7, 12]
We open the sails : [True, True, False, True, False, True]
Boat speeds are : [17.188467669562847, 20.24570723701461, 15, 24.94969596602099, 15, 21.54515226124569]
The emissions for each square are: [0, 0, 16.0, 0, 16.0, 0]
The total emissions are: 32.0
The time need for each square is: [0.29089271342400963, 0.24696593413435577, 0.3333333333333333, 0.2004032436631494, 0.3333333333333333, 0.2320707665173359]
The total time needed is: 1.6369993244055172


# Problem in 2D